In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, Flatten, Dropout, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

In [2]:
TRAIN_IMAGE_PATH = "jpeg/train/"
TEST_IMAGE_PATH = "jpeg/test/"
IMG_HEIGHT = 600
IMG_WIDTH = 600
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE
REG = 0.0005
EPOCHS=20
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),tf.keras.metrics.binary_crossentropy,
]

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [4]:
train_df["image_name"] = train_df["image_name"].apply(lambda x: TRAIN_IMAGE_PATH + x + ".jpg")
test_df["image_name"] = test_df["image_name"].apply(lambda x: TEST_IMAGE_PATH + x + ".jpg")

In [5]:
# train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=45, shuffle=True)

In [6]:
# train_df.shape, val_df.shape

In [7]:
def decode_image(filename, label=None, image_size=(IMG_WIDTH, IMG_HEIGHT)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32)/255.0
    image = tf.image.resize(image, size=image_size)
    
    if label is None:
        return image
    else:
        return image, label

In [8]:
def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    if label is None:
        return image
    else:
        return image, label

In [9]:
# def compute_class_weights(labels):
#     total_labels = labels.shape[0]
    
#     positive_labels = K.sum(labels, axis=0)/total_labels
#     negative_labels = 1 - positive_labels
#     return {0:positive_labels, 1:negative_labels}

In [10]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
image_names = train_df["image_name"].values
train_df["image_name"] = lb.fit_transform(train_df["image_name"].values)
train_df["target"] = train_df["target"].astype("int")
train_df.head()
map_name_no = dict(zip(train_df["image_name"], image_names))
y_train = train_df["target"]
x_train = train_df[["image_name"]]


over = SMOTE(random_state=45, sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
ppl = Pipeline(steps=steps)

x_train, y_train = ppl.fit_resample(x_train, y_train)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=45)
x_train["image_name"] = x_train["image_name"].apply(lambda x: map_name_no[x])
x_val["image_name"] = x_val["image_name"].apply(lambda x: map_name_no[x])

Using TensorFlow backend.


In [11]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)

In [12]:
class_weights = dict(enumerate(class_weights))

In [13]:
class_weights

{0: 0.752022342064715, 1: 1.491975544516622}

In [14]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((7809, 1), (7809,), (1953, 1), (1953,))

In [15]:
sum(y_train), sum(y_val)

(2617, 637)

In [16]:
train_dataset = (tf.data.Dataset
                 .from_tensor_slices((x_train["image_name"].values, y_train))
                 .map(decode_image, num_parallel_calls=AUTO)
                 .repeat()
                 .shuffle(512)
                 .batch(BATCH_SIZE)
                 .prefetch(AUTO)
                )

In [17]:
val_dataset = (tf.data.Dataset
               .from_tensor_slices((x_val["image_name"].values, y_val))
               .map(decode_image, num_parallel_calls=AUTO)
               .repeat()
               .shuffle(512)
               .batch(BATCH_SIZE)
               .prefetch(AUTO))

In [18]:
test_dataset = (tf.data.Dataset.from_tensor_slices((test_df.image_name))
                .map(decode_image, num_parallel_calls=AUTO)
                .batch(BATCH_SIZE))

In [19]:
lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=3,
    min_lr=0.000001,
    factor=0.5,
    verbose=1
)

In [20]:
es = EarlyStopping(monitor="val_loss", patience=10)

In [21]:
model_iv3 = InceptionV3(include_top=False, weights="imagenet", input_shape=(IMG_WIDTH, IMG_HEIGHT,3))
for layers in model_iv3.layers[:-10]:
    layers.trainable = False
model_iv3.trainable=False

model = Conv2D(filters=32, kernel_size=(3,3), data_format="channels_last", activation="relu", kernel_regularizer=l2(REG))(model_iv3.output)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filters=32, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(REG))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D()(model)
model = Dropout(0.25)(model)

model = Conv2D(filters=64, kernel_size=(3,3), data_format="channels_last", activation="relu", kernel_regularizer=l2(REG))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Conv2D(filters=64, kernel_size=(3,3), activation="relu", kernel_regularizer=l2(REG))(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = MaxPooling2D()(model)
model = Dropout(0.25)(model)


model = Flatten()(model)
model = Dense(256, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)
model = Dropout(0.5)(model)
model = Dense(64, activation="relu")(model)
model = BatchNormalization(axis=-1, center=True, scale=False)(model)

output = Dense(1, activation="sigmoid")(model)
model_iv3 = Model(inputs=model_iv3.input, outputs=output)
model_iv3.summary()
model_iv3.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4), loss="binary_crossentropy", metrics=METRICS)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 600, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 299, 299, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 299, 299, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 299, 299, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [22]:
x_val.shape

(1953, 1)

In [ ]:
history = model_iv3.fit(train_dataset, epochs=EPOCHS, callbacks=[lr, es],
                        steps_per_epoch=x_train.shape[0]//BATCH_SIZE, validation_data=val_dataset,
                        validation_steps=x_val.shape[0]//BATCH_SIZE,
                        class_weight = class_weights)

Train for 122 steps, validate for 30 steps
Epoch 1/20
122/122 [==============================] - 1006s 8s/step - loss: 0.9462 - tp: 1347.0000 - fp: 2516.0000 - tn: 2680.0000 - fn: 1265.0000 - accuracy: 0.5158 - precision: 0.3487 - recall: 0.5157 - auc: 0.5198 - binary_crossentropy: 0.8471 - val_loss: 0.8055 - val_tp: 15.0000 - val_fp: 29.0000 - val_tn: 1267.0000 - val_fn: 609.0000 - val_accuracy: 0.6677 - val_precision: 0.3409 - val_recall: 0.0240 - val_auc: 0.5183 - val_binary_crossentropy: 0.6470
Epoch 2/20
 61/122 [==============>...............] - ETA: 6:41 - loss: 0.9273 - tp: 665.0000 - fp: 1271.0000 - tn: 1346.0000 - fn: 622.0000 - accuracy: 0.5151 - precision: 0.3435 - recall: 0.5167 - auc: 0.5145 - binary_crossentropy: 0.8298

In [ ]:
probs = model_iv3.predict(test_dataset, verbose = 1)

In [ ]:
sam = pd.read_csv("sample_submission.csv")

In [ ]:
sam.head()


In [ ]:
sam["target"] = probs

In [ ]:
sam

In [ ]:
sam.to_csv("model_iv3_smote_oversampling_undersampling.csv", index=False)